In [1]:
from fastai.vision.all import *

path = untar_data(URLs.PETS)

additional_images = Path('images')
ims = get_image_files(additional_images)
for i, img in enumerate(ims):
    category = img.parent.name
    try:
        im = PILImage.create(img)
    except:
        print(f"Failed to open {img}")
        continue

    shutil.copy(src=img, dst=path/'images'/f"{category}_{i}.jpg")

failed = verify_images(path/'images')
print(f"Failed images: {failed}")
failed.map(Path.unlink)


ModuleNotFoundError: No module named 'fastai'

In [ ]:

dls = ImageDataLoaders.from_name_re(
    path, 
    get_image_files(path/'images'), 
    pat='(.+)_\d+.jpg', 
    item_tfms=RandomResizedCrop(460, min_scale=0.5),
    # item_tfms=Resize(460), 
    batch_tfms=aug_transforms(size=224, min_scale=0.75))


In [ ]:

learn = vision_learner(
    dls, 
    models.resnet50, 
    metrics=accuracy)

print(learn.dls.vocab)


In [ ]:

learn.lr_find()


In [ ]:
import optuna
from fastai.vision.all import *
import timm
from fastai.metrics import accuracy

def objective(trial):
    # Define hyperparameters to search
    # img_size = trial.suggest_categorical('img_size', [224, 256, 384])
    img_size = trial.suggest_categorical('img_size', [224, 256, 384, 512])
    # models = ['convnext_tiny', 'resnet50', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3']
    models = ['convnext_tiny','convnext_large', 'resnet50', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3']
    selected_model = trial.suggest_categorical('model', models)
    resize_method = trial.suggest_categorical('resize_method', ['Resize']) #, 'RandomResizedCrop', 'ResizeSquish', 'ResizeCrop'])
    batch_transforms = trial.suggest_categorical('batch_transforms', [True])
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    dropout = trial.suggest_float('dropout', 0.1, 0.2)
    epochs = trial.suggest_int('epochs', 3, 10)
    weight_decay = trial.suggest_float('weight_decay', 0.002, 0.03, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam'])
    
    # Print the parameters for this trial
    print(f"Trial parameters: {trial.params}")
    
    # Define transformations based on suggested hyperparameters
    if resize_method == 'Resize':
        item_tfms = [Resize(img_size)]
    elif resize_method == 'RandomResizedCrop':
        item_tfms = [RandomResizedCrop(img_size, min_scale=0.9)]
    elif resize_method == 'ResizeSquish':
        item_tfms = [Resize(img_size, method='squish')]
    else:
        item_tfms = [Resize(img_size, method='crop')]
    
    batch_tfms = [
        *aug_transforms(  # Apply augmentations cautiously
            size=img_size,
            flip_vert=False,  # Avoid flipping vertically, important for medical data
            max_rotate=30,    # Limit rotations to ±10 degrees
            max_zoom=1.5,     # Slight zooming
            max_lighting=0.4, # Light contrast/brightness adjustment
            p_affine=0.5,     # Probability of affine transform
            p_lighting=0.3    # Probability of lighting transform
        ),
        Normalize.from_stats(*imagenet_stats)  # Normalize based on ImageNet mean/std
    ] if batch_transforms else [Normalize.from_stats(*imagenet_stats)]

    # Create DataLoaders
    dls = ImageDataLoaders.from_name_re(
        path, 
        get_image_files(path/'images'), 
        pat='(.+)_\d+.jpg', 
        item_tfms=item_tfms,
        batch_tfms=batch_tfms,
        bs=batch_size,
        shuffle_train=True)  # Shuffle the training data

    # Check if the DataLoader is loaded correctly
    dls.show_batch(max_n=4, figsize=(8, 8))

    the_model = timm.create_model(selected_model, pretrained=True, num_classes=dls.c)
    # Define the model with dropout
    def create_model(the_model=the_model, dropout=0.5):
        # Get the number of features from the classifier layer
        num_ftrs = the_model.get_classifier().in_features
        
        # Replace the classifier with dropout and a linear layer
        # The classifier layer is replaced regardless of its internal name
        the_model.reset_classifier(num_classes=dls.c)  # Resets the classifier dynamically
        the_model.classifier = nn.Sequential(
            nn.Dropout(p=dropout),  # Add dropout with the rate defined in your hyperparameter search
            nn.Linear(num_ftrs, dls.c)
        )
        
        return the_model
    
    # Create the learner
    patience = 3
    learn = Learner(
        dls, 
        create_model(the_model, dropout), 
        metrics=accuracy,
        wd=weight_decay,
        # cbs=[EarlyStoppingCallback(monitor='accuracy', patience=patience)]
        ).to_bf16()
    
    learn.lr_find(suggest_funcs=(slide, valley))
    
    # Choose optimizer
    if optimizer_name == 'Adam':
        learn.opt_func = Adam
    elif optimizer_name == 'SGD':
        learn.opt_func = SGD
    elif optimizer_name == 'RMSprop':
        learn.opt_func = RMSProp
    
    # Train the model
    learn.fit(epochs, lr)
    
    # Initialize accuracy
    accuracy_result = 0.0
 
    # Evaluate the model
    try:
        # Evaluate the model
        validation_results = learn.validate()
        if validation_results and len(validation_results) > 1:
            accuracy_result = validation_results[1]
    except Exception as e:
        print(f"Validation failed: {e}")
        accuracy_result = 0.0  # Default value if validation fails
 
    return accuracy_result

# Create a study and optimize the objective function
study = optuna.create_study(
    storage="sqlite:///db.sqlite3",
    direction='maximize',
    study_name='Pet Classification - 1')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print(study.best_params)

In [ ]:

learn.fine_tune(1)
learn.path = Path('.')
learn.export()

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

interp.plot_top_losses(50, nrows=10)